In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import scipy
from metpy.units import units
import metpy.calc as mpcalc
import math

import sys
sys.path.append('/Users/clintonalden/Documents/Research/summa_work/')
from utils import lw_clr
from utils import forcing_filler as ff


In [14]:
# Read in csv data from Harts Pass SNOTEL
# Subset after October 17 as the data is poor before then
harts = pd.read_csv('hrpw1_WY24.csv', skiprows=10)
harts = harts[harts['Date_Time'] > '2023-10-17'] 

# Load template forcing file with correct attributes
template = xr.open_dataset('../summa_forcing_template.nc')

# Pull in only necessary columns and rename to match SUMMA formatting
harts = harts[['Date_Time', 'air_temp_set_1', 'wind_speed_set_1', 'solar_radiation_set_1', 'precip_accum_set_1']]
replace = {'air_temp_set_1':'airtemp','wind_speed_set_1':'windspd', 'solar_radiation_set_1':'SWRadAtm', 
           'precip_accum_set_1':'accppt', 'Date_Time':'time'}
harts.rename(columns=replace, inplace=True)

# Convert time column to pandas datetime format
harts['time'] = pd.to_datetime(harts['time'])

# Use cummax to fix the issues with SNOTEL hourly data
harts['accppt'] = harts['accppt'].cummax()

# Convert precip from mm hr-1 to kg m-2 s-1
harts['pptrate'] = harts['accppt'].diff()*3600

# Generate relative humidity assuming T_d is overnight low temperature
ff.fill_rel_hum(harts)

# Convert temperature from C to K
harts['airtemp'] = harts['airtemp'] + 273.15

# Set NaN SW radiation values to 0
harts['SWRadAtm'] = harts['SWRadAtm'].fillna(0)

# Set NaN precip values to 0
harts['pptrate'] = harts['pptrate'].fillna(0)

# Generate pressure from hypsometric equation and site elevation (1981m)
ff.fill_pressure(harts, 1981)

# Generate specific humidity
ff.fill_spec_hum(harts)

# Generate longwave radiation
harts['LWRadAtm'] = lw_clr.dilleyobrien1998(harts['airtemp'], harts['rh'])

# Drop unnecessary columns
harts = harts.drop(columns=['accppt', 'rh'])

# Set time as index
harts.set_index('time', inplace=True)



In [17]:
# Convert dataframe to xarray
dsx = harts.to_xarray()

# Loop through variables and add attributes from template forcing file
for data_var in dsx:
    dsx[data_var].attrs = template[data_var].attrs
    
# Add hru dimension
dsx = dsx.expand_dims(dim={'hru':1})

# Add gap-filled and datastep variables
dsx['gap_filled'] = xr.DataArray(np.ones((1,dsx.time.shape[0])),dims = ['hru','time'])
dsx['data_step'] = 3600 # 3600 seconds for 1hr timesteps

# Transpose gap filled variable to match dimensions with the rest
dsx['gap_filled'] = dsx['gap_filled'].T

# Convert all to float64
for var in dsx.data_vars:
    dsx[var] = dsx[var].astype(np.float64)
    
# Set hruID based on template
dsx.to_netcdf('./harts_SNTL_WY24.nc',
              engine='h5netcdf')


In [11]:
dsx

<xarray.Dataset> Size: 260kB
Dimensions:     (time: 3607, hru: 1)
Coordinates:
  * time        (time) object 29kB '2023-10-17T00:00:00Z' ... '2024-03-15T12:...
Dimensions without coordinates: hru
Data variables:
    airtemp     (hru, time) float64 29kB 277.6 276.2 276.0 ... 276.3 275.8 276.0
    windspd     (hru, time) float64 29kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    SWRadAtm    (hru, time) float64 29kB 49.85 0.87 0.0 0.0 ... 0.0 0.0 0.0 0.0
    pptrate     (hru, time) float64 29kB 0.0 1.829e+04 0.0 0.0 ... 0.0 0.0 0.0
    airpres     (hru, time) float64 29kB 8.71e+04 8.703e+04 ... 8.702e+04
    spechum     (hru, time) float64 29kB 0.005594 0.005401 ... 0.004635 0.004653
    LWRadAtm    (hru, time) float64 29kB 255.0 250.2 248.8 ... 245.6 243.8 244.7
    gap_filled  (time, hru) float64 29kB 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    data_step   float64 8B 3.6e+03
    hruId       (hru) int32 4B 1001